In [22]:
import mlflow
import torch
import os
import sys

# Add the parent directory of 'src' to the sys.path
parent_dir = os.path.dirname(
    os.getcwd()
)  # Get the parent directory of the current working directory
sys.path.append(parent_dir)
mlflow.set_tracking_uri(os.path.join(os.path.dirname(os.getcwd()), "mlruns"))

# Now you can import your modules
import src.pipeline.rankme_evaluator as RankMe
import src.pipeline.finetune as finetune
from src.models.encoder_projector import EncoderProjector
from src.utils.load_dataset import load_dataset

In [34]:
from mlflow import MlflowClient
import torchvision.transforms as transforms
from src.utils.pytorch_device import get_device

device = get_device("mps")

# Filter for runs where the tag 'status' is set to 'succeeded'
filter_string = "tags.status = 'succeeded'"

runs = MlflowClient().search_runs(
    experiment_ids="0",  # gött att det är en sträng
    filter_string=filter_string,
    max_results=1,
)

# Required for evaluation
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize((32, 32)),
        transforms.Normalize(
            mean=[0.5071, 0.4867, 0.4408], std=[0.2675, 0.2565, 0.2761]
        ),
    ]
)
dataset_name = "CIFAR100"
dataset = load_dataset(dataset_name=dataset_name, transform=transform)

# oklart om nödvändigt
mlflow.end_run()

for run in runs:
    run_id = run.info.run_id
    model_uri = f"runs:/{run_id}/pretraining_model/"
    model = mlflow.pytorch.load_model(model_uri)
    rank = RankMe.rank_me(model=model, dataset=dataset, device=device)
    # update the run with the rank as a metric
    # den här raden klagar på att "the run is already active" >:(
    with mlflow.start_run(run_id=run_id) as run:
        mlflow.log_metric(f"{dataset_name}/rank", float(rank))
        mlflow.end_run()

Files already downloaded and verified
<Run: data=<RunData: metrics={'learning_rate': 0.1, 'loss': 21.292160034179688}, params={'batch_size': '64',
 'cov_coeff': '1.0',
 'dataset': 'cifar10',
 'dataset_dir': '_datasets',
 'device': 'mps',
 'epochs': '1',
 'featdim': '128',
 'logfile': 'logs/rankme.log',
 'lr': '0.1',
 'model': 'resnet18',
 'model_dir': '_models',
 'sim_coeff': '25.0',
 'std_coeff': '25.0',
 'temperature': '0.07',
 'trainer': 'vicreg',
 'verbosity': '0',
 'weight_decay': '1e-6',
 'workers': '8'}, tags={'mlflow.gitRepoURL': 'git@github.com:johnwikman/rankme-reprod.git',
 'mlflow.log-model.history': '[{"run_id": "955dbd346f8c48efac438bb00dfa4556", '
                             '"artifact_path": "pretraining_model", '
                             '"utc_time_created": "2023-11-27 '
                             '22:31:34.027818", "flavors": {"pytorch": '
                             '{"model_data": "data", "pytorch_version": '
                             '"2.1.0", "code": n